In [78]:
%config IPCompleter.greedy=True

In [79]:
%config Completer.use_jedi = True

<h3><center>Kinopoisk TOP-250 Movies Page Parsing<center><br>
    using Requests, BeautifulSoup4, Selenium

Natalia Suchkova - 25.01.2023

In [107]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium.common.exceptions import NoSuchElementException


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from fake_useragent import UserAgent

In [2]:
from lxml import etree

In [3]:
import pandas as pd
import re
import unicodedata

import time

### Тестовый парсинг на одном эксземпляре

In [38]:
url = 'https://www.kinopoisk.ru/lists/movies/top250/?page=1'

In [39]:
r = requests.get(url)
r.text

'<!DOCTYPE html><html><head><meta name="viewport" content="width=device-width"/><meta charSet="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta name="theme-color" content="#1f1f1f" data-tid="57f72b5"/><meta name="apple-mobile-web-app-capable" content="yes" data-tid="57f72b5"/><meta name="apple-mobile-web-app-status-bar-style" content="black" data-tid="57f72b5"/><meta name="apple-mobile-web-app-title" content="Кинопоиск" data-tid="57f72b5"/><meta name="apple-itunes-app" content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5"/><meta name="application-name" content="Кинопоиск" data-tid="57f72b5"/><meta property="fb:app_id" content="121953784483000" data-tid="57f72b5"/><meta property="fb:pages" content="152308956519" data-tid="57f72b5"/><meta property="og:site_name" content="Кинопоиск" data-tid="57f72b5"/><meta name="msapplication-TileColor" content="#000" data-tid="57f72b5"/><meta name="msapplication-TileImage" content="https://yastatic.net/s

In [40]:
soup = BeautifulSoup(r.text, features='lxml')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="width=device-width" name="viewport"/>
  <meta charset="utf-8"/>
  <title data-tid="57f72b5">
   250 лучших фильмов — Кинопоиск
  </title>
  <meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/>
  <meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/>
  <meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/>
  <meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/>
  <meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/>
  <meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/>
  <meta content="152308956519" data-tid="57f72b5" property="fb:pages"/>
  <meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/>
  <meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/>
  <meta content="ht

In [41]:
link = 'https://www.kinopoisk.ru' + soup.find('div', class_='styles_upper__j8BIs').find('a', class_='base-movie-main-info_link__YwtP1').get('href')

In [42]:
link

'https://www.kinopoisk.ru/film/435/'

In [43]:
name = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
name

'Зеленая миля'

In [44]:
name_eng = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='desktop-list-main-info_secondaryTitle__ighTt').text
name_eng

'The Green Mile'

In [45]:
for i in soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP'):
    print(i.get_text(strip=True))
   # print(unicodedata.normalize("NFKD", i.text))

США • драма  Режиссёр: Фрэнк Дарабонт
В ролях: Том Хэнкс, Дэвид Морс


In [46]:
soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')

[<span class="desktop-list-main-info_truncatedText__IMQRP">США • драма  Режиссёр: Фрэнк Дарабонт</span>,
 <span class="desktop-list-main-info_truncatedText__IMQRP">В ролях: Том Хэнкс, Дэвид Морс</span>]

In [47]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile("Режиссёр: (.*)$")
director = regexp.search(string).group(1)
director 

'Фрэнк Дарабонт'

In [48]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile('(\w+) •')
country = regexp.search(string).group(1)
country 

'США'

In [49]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile('• (.*)$')
genre = regexp.search(string).group(1)
regexp = re.compile('[^  ]*')
genre = regexp.search(genre).group(0)
genre

'драма'

In [50]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text)
print(string)
year = string.split(', ')[1]
minutes_time = re.compile('[0-9]+').search(string.split(', ')[2]).group(0)
print(year, minutes_time)

, 1999, 189 мин.
1999 189


In [51]:
rating = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text
rating

'9.2'

### Объединенный парсинг всех фильмов на 1 странице (из 5ти)

Отметим, что есть отличия при парсинге российских и иностранных фильмов, так как у первых отсутствует название на английском языке. Английское название, год выпуска и продолжительность находятся в одном классе и подклассе

`div class="desktop-list-main-info_secondaryTitleSlot__mc0mI">`

   `<span class="desktop-list-main-info_secondaryTitle__ighTt">The Green Mile</span>`
    
   `<span class="desktop-list-main-info_secondaryText__M_aus">, 1999, 189&nbsp;мин.</span>`
    
`</div`

In [52]:
films = soup.find_all('div', class_='styles_upper__j8BIs')
len(films)

50

In [70]:
link = []
name = []
name_eng = []
rating = []
director = []
country = []
genre = []
year = []
minutes_time = []

num_pages = 5

In [72]:
for page in range(1, num_pages + 1):
    
    url = f'https://www.kinopoisk.ru/lists/movies/top250/?page={page}'
    print(url)
    r = requests.get(url)
    time.sleep(40) 
    soup = BeautifulSoup(r.content, features='lxml')
    time.sleep(60)  # for captcha problem stabilization
    films = soup.find_all('div', class_='styles_upper__j8BIs')
    time.sleep(60)
    print(len(films))  # monitoring url response

    for film in films:
        link.append('https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href'))
        name.append(film.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text)

        rating.append(float(
                          film.find('span', 
                          class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text)
                         )

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        director.append(re.compile("Режиссёр: (.*)$").search(string).group(1))
        
        time.sleep(1)
        
        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        country.append(re.compile('(\w+) •').search(string).group(1))

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        genre.append(re.compile('[^  ]*').search(re.compile('• (.*)$').search(string).group(1)).group(0))

        time.sleep(2)
        
        data_block = film.find('div', class_='desktop-list-main-info_secondaryTitleSlot__mc0mI').find_all('span')
        
        if len(data_block) == 2:

            name_eng.append(data_block[0].text)
            string = unicodedata.normalize("NFKD", data_block[1].text)
            year.append(int(string.split(', ')[1]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[2]).group(0)))

        else:
            
            name_eng.append('---')
            string = unicodedata.normalize("NFKD", data_block[0].text)
            year.append(int(string.split(', ')[0]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[1]).group(0)))

            
    data = {'Kinopoisk_Link': link, 'Name': name, 'Name_ENG': name_eng,
            'Year': year, 'Country': country, 'Genre': genre,
            'Director': director, 'Length': minutes_time, 'Rating': rating}
    
    time.sleep(60)

https://www.kinopoisk.ru/lists/movies/top250/?page=1
50
https://www.kinopoisk.ru/lists/movies/top250/?page=2
50
https://www.kinopoisk.ru/lists/movies/top250/?page=3
50
https://www.kinopoisk.ru/lists/movies/top250/?page=4
50
https://www.kinopoisk.ru/lists/movies/top250/?page=5
0


In [74]:
data = pd.DataFrame(data)

In [75]:
data

,Kinopoisk_Link,Name,Name_ENG,Year,Country,Genre,Director,Length,Rating
0,https://www.kinopoisk.ru/film/435/,Зеленая миля,The Green Mile,1999,США,драма,Фрэнк Дарабонт,189,9.2
1,https://www.kinopoisk.ru/film/329/,Список Шиндлера,Schindler's List,1993,США,драма,Стивен Спилберг,195,9.1
2,https://www.kinopoisk.ru/film/326/,Побег из Шоушенка,The Shawshank Redemption,1994,США,драма,Фрэнк Дарабонт,142,9.0
3,https://www.kinopoisk.ru/film/448/,Форрест Гамп,Forrest Gump,1994,США,драма,Роберт Земекис,142,9.0
4,https://www.kinopoisk.ru/film/679486/,Тайна Коко,Coco,2017,США,мультфильм,Ли Анкрич,105,8.9
...,...,...,...,...,...,...,...,...,...
195,https://www.kinopoisk.ru/film/837530/,Моана,Moana,2016,США,мультфильм,Рон Клементс,107,8.2
196,https://www.kinopoisk.ru/film/689066/,Стражи Галактики,Guardians of the Galaxy,2014,США,фантастика,Джеймс Ганн,121,8.2
197,https://www.kinopoisk.ru/film/751/,Военный ныряльщик,Men of Honor,2000,США,драма,Джордж Тиллман мл.,129,8.2
198,https://www.kinopoisk.ru/film/338/,Звёздные войны: Эпизод 5 — Империя наносит отв...,Star Wars: Episode V - The Empire Strikes Back,1980,США,фантастика,Ирвин Кершнер,124,8.2


In [141]:
browser = webdriver.Chrome(r'C:\Users\User\Desktop\chromedriver.exe')
url = 'https://www.kinopoisk.ru/lists/movies/top250/'

In [142]:
browser.get(url)

In [143]:
# пробуем листать страницы сайта с селениумом
while True:
    try:
    #     next_page_button = browser.find_element_by_css_selector('#__next > div.styles_root__goB3B > div.styles_middleContainer__vJjLN.styles_baseContainer__bGRTT > div.styles_mainContainer__59YLl > div.styles_contentSlot__h_lSN > main > div.styles_root__AT6_5.styles_root__RoFSb > a.styles_end__aEsmB.styles_start__UvE6T').click()
        next_page_button = browser.find_element_by_xpath('//a[@title="Вперед"]')
        next_page_button.click()
        print(browser.current_url)
    
        # !!!! селениум не может найти по всему классу, видит название класса только до пробела,
        # !!!! потом для него следующий уже второй класс и тд. а он как и суп при поиске останавливается на первом
    except NoSuchElementException:
        print('No Such Element on the Page')
        break

url = browser.current_url
print('showcaptcha' in url)
browser.quit()

https://www.kinopoisk.ru/lists/movies/top250/?page=2
https://www.kinopoisk.ru/lists/movies/top250/?page=3
https://www.kinopoisk.ru/lists/movies/top250/?page=4
https://www.kinopoisk.ru/lists/movies/top250/?page=5
No Such Element on the Page
https://www.kinopoisk.ru/lists/movies/top250/?page=5 False


In [161]:
def wait_until(somepredicate, timeout, period=0.5):
    mustend = time.time() + timeout
    while time.time() < mustend:
        if somepredicate(): 
            return True
        else:
            time.sleep(period)
    return False

In [201]:
def check_captcha(browser, url, page_soup):
    if 'captcha' in url or page_soup.find('span', class_='Text_typography_body-long-m'): 
#     \
#     or 'Нам очень жаль' in \
#     BeautifulSoup(requests.get(url).content,
#                   features='lxml').find('span', \
#     class_='Text_typography_body-long-m').text:
        print("Captcha's here!!!!")
        box_1 = browser.find_element_by_xpath('//input[@type="submit"]')
        box_1.click()
        time.sleep(60)

        while 'captcha' in browser.current_url:
            pass
    return browser.current_url

In [202]:
link = []
name = []
name_eng = []
rating = []
director = []
country = []
genre = []
year = []
minutes_time = []

In [203]:
browser = webdriver.Chrome(r'C:\Users\User\Desktop\chromedriver.exe')
url = 'https://www.kinopoisk.ru/lists/movies/top250/'
browser.get(url)
time.sleep(5)

while True:
    
    url = browser.current_url
    time.sleep(3)
   
    url = check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    print(url)
    
    r = requests.get(url)
    time.sleep(2)  # for captcha problem stabilization
    check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    
    soup = BeautifulSoup(r.content, features='lxml')
    print(soup)
    time.sleep(3)  
    films = soup.find_all('div', class_='styles_upper__j8BIs')
    print(len(films))  # monitoring url response

    for film in films:
        link.append('https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href'))
        name.append(film.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text)

        rating.append(float(
                          film.find('span', 
                          class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text)
                         )

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        director.append(re.compile("Режиссёр: (.*)$").search(string).group(1))
        
        time.sleep(2)
        
        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        country.append(re.compile('(\w+) •').search(string).group(1))

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        genre.append(re.compile('[^  ]*').search(re.compile('• (.*)$').search(string).group(1)).group(0))

        time.sleep(2)
        
        data_block = film.find('div', class_='desktop-list-main-info_secondaryTitleSlot__mc0mI').find_all('span')
        
        if len(data_block) == 2:

            name_eng.append(data_block[0].text)
            string = unicodedata.normalize("NFKD", data_block[1].text)
            year.append(int(string.split(', ')[1]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[2]).group(0)))

        else:
            
            name_eng.append('---')
            string = unicodedata.normalize("NFKD", data_block[0].text)
            year.append(int(string.split(', ')[0]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[1]).group(0)))
    
    data = {'Kinopoisk_Link': link, 'Name': name, 'Name_ENG': name_eng,
            'Year': year, 'Country': country, 'Genre': genre,
            'Director': director, 'Length': minutes_time, 'Rating': rating}

    
    try:  # here we navigate the pages
        next_page_button = browser.find_element_by_xpath('//a[@title="Вперед"]')
        next_page_button.click()
        time.sleep(3)

    except NoSuchElementException:
#         browser.quit()
        break


https://www.kinopoisk.ru/lists/movies/top250/
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta content="https:/

50
https://www.kinopoisk.ru/lists/movies/top250/?page=2
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta conten

50
https://www.kinopoisk.ru/lists/movies/top250/?page=3
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta conten

50
https://www.kinopoisk.ru/lists/movies/top250/?page=4
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style></head><body><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div 

0
https://www.kinopoisk.ru/lists/movies/top250/?page=5
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta content

50


In [204]:
data = pd.DataFrame(data)

In [205]:
data

,Kinopoisk_Link,Name,Name_ENG,Year,Country,Genre,Director,Length,Rating
0,https://www.kinopoisk.ru/film/435/,Зеленая миля,The Green Mile,1999,США,драма,Фрэнк Дарабонт,189,9.2
1,https://www.kinopoisk.ru/film/329/,Список Шиндлера,Schindler's List,1993,США,драма,Стивен Спилберг,195,9.1
2,https://www.kinopoisk.ru/film/326/,Побег из Шоушенка,The Shawshank Redemption,1994,США,драма,Фрэнк Дарабонт,142,9.0
3,https://www.kinopoisk.ru/film/448/,Форрест Гамп,Forrest Gump,1994,США,драма,Роберт Земекис,142,9.0
4,https://www.kinopoisk.ru/film/679486/,Тайна Коко,Coco,2017,США,мультфильм,Ли Анкрич,105,8.9
...,...,...,...,...,...,...,...,...,...
195,https://www.kinopoisk.ru/film/461939/,Линкольн для адвоката,The Lincoln Lawyer,2011,США,криминал,Брэд Фурман,114,8.2
196,https://www.kinopoisk.ru/film/81041/,Алеша Попович и Тугарин Змей,---,2004,Россия,мультфильм,Константин Бронзит,79,8.2
197,https://www.kinopoisk.ru/film/81555/,Загадочная история Бенджамина Баттона,The Curious Case of Benjamin Button,2008,США,драма,Дэвид Финчер,166,8.2
198,https://www.kinopoisk.ru/film/333/,Звёздные войны: Эпизод 4 — Новая надежда,Star Wars,1977,США,фантастика,Джордж Лукас,121,8.1


In [ ]:


options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(chrome_options=options, executable_path=r'.\chromedriver.exe')
driver.get("https://www.google.co.in")
driver.quit()